# Buy and Hold Strategy using MA120

## Background



To-Do:


---
## 0. Imports

In [31]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../functions_library')

from functions import ROC, model_selection, Backtesting

#import sklearn 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import sklearn.ensemble

# Classification Metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc #plot_roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# ML models 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

# libraries for Shapely analysis
import shap 

# Visualization libraries
import matplotlib.pyplot as plt
import hvplot.pandas

seed = 42

---
## 1. Data Loading

Read the CSV file into Pandas DataFrame

In [32]:
# Import the OHLCV dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../Resources/ETHUSDT-1h-data.csv"), 
    index_col="timestamp", 
    infer_datetime_format=True, 
    parse_dates=True
)

trading_df = trading_df.drop(["close_time","quote_av","trades","tb_base_av","tb_quote_av","ignore"], axis =1)

# Review the DataFrame
trading_df.head()

,open,high,low,close,volume
timestamp,,,,,
2017-08-17 04:00:00,301.13,302.57,298.0,301.61,125.66877
2017-08-17 05:00:00,301.61,303.28,300.0,303.10,377.67246
2017-08-17 06:00:00,302.40,304.44,301.9,302.68,303.86672
2017-08-17 07:00:00,302.68,307.96,302.6,307.96,754.74510
2017-08-17 08:00:00,307.95,309.97,307.0,308.62,150.75029


In [33]:
import pandas as pd

# Load the ETHUSD OHLCV data
eth_df = df

# resample to 24H frequency and calculate OHLCV
df_resampled = df.resample('24H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

# Filter for the time range of interest
start_date = '2017-08-18'
end_date = '2023-01-22'
eth_df = eth_df.loc[start_date:end_date]

# Calculate the total amount of ETH that can be bought with $10
eth_df['eth_per_10usd'] = (10 / eth_df['close']) * (1 - 0.01)

# Calculate the amount of ETH that can be bought each day at 12:00 pm
eth_df['eth_bought'] = 0
for i, row in eth_df.iterrows():
    eth_df.at[i, 'eth_bought'] = 10 * eth_df.at[i, 'eth_per_10usd']

# Calculate the cumulative amount of ETH bought over time
eth_df['eth_cumulative'] = eth_df['eth_bought'].cumsum()

# Calculate the value of the ETH holdings in USD
eth_df['usd_value'] = eth_df['eth_cumulative'] * eth_df['close']

# Plot the cumulative ETH holdings value
eth_df['usd_value'].hvplot(figsize=(10, 6), title='Cumulative Value of $10 Daily ETH Purchases')

:Curve   [timestamp]   (usd_value)

In [41]:
import finta as ft
from finta import TA
import talib

ohlcv_df = eth_df

# List of time periods to use for Moving Averages calculation
timeperiods = [5,7,14,20,30,50,70,100,150,200]

feat_df = ohlcv_df.copy()

# Calculate SMAs and add them to the DataFrame
for t in timeperiods:
    #tsma = TA.SMA(df, t).shift(1)
    sma = TA.SMA(df, t)
    feat_df[f'SMA_{t}'] = sma
    

In [42]:
feat_df = feat_df.dropna()
display(feat_df)

,open,high,low,close,volume,purchase_value,quantity,investment_value,cumulative_investment_value,eth_per_10usd,...,eth_sma120_usd_value,SMA_5,SMA_7,SMA_14,SMA_20,SMA_30,SMA_50,SMA_70,SMA_150,SMA_200
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-08-28,287.71,288.00,283.64,286.59,12768.01273,9.90099,0.034548,9.90099,3712.871287,0.034544,...,6386.674480,278.310,277.850000,284.999286,297.4650,337.929000,386.5382,410.291714,504.144667,560.45195
2018-08-29,295.44,296.10,294.49,295.52,10829.33474,9.90099,0.033504,9.90099,3722.772277,0.033500,...,6684.680038,282.722,280.267143,285.912857,294.4035,332.338000,383.0272,406.976714,503.480133,557.48200
2018-08-30,288.35,291.00,288.18,290.23,9750.46108,9.90099,0.034114,9.90099,3732.673267,0.034111,...,6664.019923,284.450,282.471429,286.412857,290.6605,326.839667,380.0326,403.464571,502.857933,554.74555
2018-08-31,283.90,284.27,280.51,280.83,9544.32327,9.90099,0.035256,9.90099,3742.574257,0.035253,...,6547.184939,285.370,283.524286,285.598571,288.3185,321.813000,376.8014,399.980714,502.182200,552.04115
2018-09-01,281.46,284.80,281.34,284.19,7196.75087,9.90099,0.034839,9.90099,3752.475248,0.034836,...,6724.518954,287.472,283.895714,283.130714,286.5475,317.205667,373.8520,397.298857,501.305133,549.13490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-18,1565.56,1574.58,1562.93,1570.59,13596.59370,9.90099,0.006304,9.90099,19594.059406,0.006303,...,461195.267102,1552.506,1511.971429,1398.957857,1340.2090,1296.815667,1282.6100,1260.200857,1365.129400,1405.75255
2023-01-19,1511.44,1523.25,1509.15,1516.26,18982.55220,9.90099,0.006530,9.90099,19603.960396,0.006529,...,445241.556164,1547.634,1526.404286,1417.692857,1356.0205,1308.285333,1287.4398,1265.488286,1364.502733,1408.01805
2023-01-20,1551.09,1554.68,1546.00,1548.21,9966.80790,9.90099,0.006395,9.90099,19613.861386,0.006394,...,454623.501028,1551.678,1546.714286,1438.588571,1373.5605,1319.594333,1292.5998,1269.081000,1364.019867,1410.39505


In [52]:
import pandas as pd

# Load the ETHUSD OHLCV data
eth_df = feat_df

# resample to 24H frequency and calculate OHLCV
df_resampled = df.resample('24H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

# Filter for the time range of interest
start_date = '2022-11-25'
end_date = '2023-01-22'
eth_df = eth_df.loc[start_date:end_date]

# Calculate the total amount of ETH that can be bought with $10
eth_df['eth_per_10usd'] = (10 / eth_df['close']) * (1 - 0.01)

# Calculate the amount of ETH that can be bought each day at 12:00 pm
eth_df['eth_bought'] = 0
eth_df['eth_sma120_bought'] = 0
for i, row in eth_df.iterrows():
    eth_df.at[i, 'eth_bought'] = 10 * eth_df.at[i, 'eth_per_10usd']
    if eth_df.at[i, 'close'] < eth_df.at[i, 'SMA_50']: 
        eth_df.at[i, 'eth_sma120_bought'] = 10 * eth_df.at[i, 'eth_per_10usd']

# Calculate the cumulative amount of ETH bought over time
eth_df['eth_cumulative'] = eth_df['eth_bought'].cumsum()

# Calculate the value of the ETH holdings in USD
eth_df['usd_value'] = eth_df['eth_cumulative'] * eth_df['close']

# Plot the cumulative ETH holdings value
eth_df['usd_value'].hvplot(figsize=(10, 6), title='Cumulative Value of $10 Daily ETH Purchases')        
        
# Calculate the cumulative amount of ETH bought over time
eth_df['eth_sma120_cumulative'] = eth_df['eth_sma120_bought'].cumsum()

# Calculate the value of the ETH holdings in USD
eth_df['eth_sma120_usd_value'] = eth_df['eth_sma120_cumulative'] * eth_df['close']

# Plot the cumulative ETH holdings value
eth_df[['usd_value','eth_sma120_usd_value']].hvplot(figsize=(10, 6), title='Cumulative Value of $10 Daily ETH Purchases')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)